# Aprendizaje de Máqina - Predicción de precios de casas
- Source: Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project

# Imports

#### Python libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 50)

import numpy as np

import sys

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn import datasets
from sklearn.model_selection import train_test_split

#### Ancillary modules

In [2]:
%load_ext autoreload
%autoreload 2

sys.path.append("../utils")
from houses_funcs import *
from houses_params import *

# Import datasets

In [3]:
## Sample submission
df_samplesub = pd.read_csv("../data/sample_submission.csv")
df_samplesub

,id,SalePrice
0,1,180401.23
1,2,180401.23
2,3,180401.23
3,4,180401.23
4,5,180401.23
...,...,...
1198,1199,180401.23
1199,1200,180401.23
1200,1201,180401.23
1201,1202,180401.23


In [4]:
## Main dataset
df_test = pd.read_csv("../data/casas_prueba.csv")
df_test

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,...,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,id
0,60,RL,NaN,10316,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,None,...,1,TA,Attchd,2000.0,RFn,3,839,TA,TA,Y,0,184,0,0,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal,1
1,120,RL,43.0,7052,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,TwnhsE,1Story,7,5,2005,2005,Gable,CompShg,VinylSd,VinylSd,Stone,...,1,Gd,Attchd,2005.0,RFn,2,484,TA,TA,Y,192,36,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,2
2,20,RL,76.0,8243,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1961,1961,Gable,CompShg,VinylSd,VinylSd,BrkFace,...,1,Fa,Detchd,1985.0,Fin,2,784,TA,TA,Y,170,0,0,0,0,0,NaN,GdPrv,NaN,0,2,2007,WD,Normal,3
3,60,RL,NaN,18275,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,2Story,7,8,1962,1998,Gable,CompShg,Plywood,Plywood,None,...,2,Gd,Attchd,1962.0,RFn,2,441,TA,TA,Y,520,102,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Normal,4
4,20,RL,100.0,15263,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,ClearCr,Feedr,Norm,1Fam,1Story,5,5,1959,1959,Gable,CompShg,HdBoard,HdBoard,BrkFace,...,2,Gd,Attchd,1959.0,Unf,1,365,TA,TA,Y,0,132,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,20,RL,90.0,10454,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1957,1957,Hip,CompShg,Plywood,Plywood,Stone,...,0,NaN,Detchd,1957.0,Unf,1,308,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,Normal,1199
1199,120,RL,51.0,3635,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2007,2007,Hip,CompShg,VinylSd,VinylSd,BrkFace,...,1,TA,Attchd,2007.0,RFn,3,660,TA,TA,Y,143,20,0,0,0,0,NaN,NaN,NaN,0,5,2009,WD,Normal,1200
1200,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,...,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,1201
1201,20,RL,60.0,11664,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Artery,Norm,1Fam,1Story,6,5,1948,1950,Gable,CompShg,MetalSd,MetalSd,BrkFace,...,1,Gd,Detchd,1948.0,Unf,1,240,TA,TA,Y,0,130,0,0,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal,1202


In [5]:
## Main dataset
df_train = pd.read_csv("../data/casas_entrena.csv")
df_train

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,...,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,120,RM,NaN,3072,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2004,2004,Hip,CompShg,VinylSd,VinylSd,BrkFace,...,1,TA,Attchd,2004.0,Fin,2,388,TA,TA,Y,143,20,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Normal,225000
1,120,RL,53.0,4045,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2006,2006,Hip,CompShg,VinylSd,VinylSd,BrkFace,...,1,Gd,Attchd,2006.0,Fin,3,648,TA,TA,Y,161,20,0,0,0,0,NaN,NaN,NaN,0,10,2006,New,Partial,246578
2,120,RL,43.0,3013,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2005,2005,Gable,CompShg,VinylSd,VinylSd,BrkFace,...,1,Gd,Attchd,2005.0,Fin,2,440,TA,TA,Y,142,20,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Normal,213490
3,120,RL,53.0,3922,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2006,2007,Gable,CompShg,WdShing,Wd Shng,BrkFace,...,1,Gd,Attchd,2007.0,Fin,3,648,TA,TA,Y,144,16,0,0,0,0,NaN,NaN,NaN,0,6,2007,New,Partial,172500
4,120,RL,NaN,3196,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,8,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,...,1,TA,Attchd,2003.0,Fin,2,400,TA,TA,Y,143,20,0,0,0,0,NaN,NaN,NaN,0,5,2006,WD,Normal,215000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,20,RL,80.0,12000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Veenker,Norm,Norm,1Fam,1Story,7,6,1980,1980,Hip,CompShg,VinylSd,MetalSd,BrkFace,...,1,TA,Attchd,1980.0,RFn,2,546,Gd,TA,Y,180,16,0,0,0,0,NaN,NaN,NaN,0,3,2007,WD,Normal,255000
1463,20,RL,NaN,14694,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Veenker,Norm,Norm,1Fam,1Story,8,9,1977,2008,Gable,CompShg,MetalSd,MetalSd,BrkFace,...,1,Gd,Attchd,1977.0,Fin,2,642,TA,TA,Y,501,120,0,225,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal,318750
1464,60,RL,NaN,19522,Pave,NaN,IR1,Bnk,AllPub,Inside,Gtl,Veenker,Norm,Norm,1Fam,2Story,7,5,1990,1990,Gable,CompShg,HdBoard,HdBoard,BrkFace,...,1,TA,Attchd,1990.0,RFn,2,564,TA,TA,Y,0,99,0,0,182,0,NaN,NaN,NaN,0,2,2007,WD,Normal,300000
1465,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,...,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [6]:
df_train["Lot Frontage"]

0        NaN
1       53.0
2       43.0
3       53.0
4        NaN
        ... 
1462    80.0
1463     NaN
1464     NaN
1465    80.0
1466     NaN
Name: Lot Frontage, Length: 1467, dtype: float64

---

# EDA + GEDA

#### How does the "Sale Type" attribute affects the Sale Price?

In [ ]:
dfx = df_train.copy()

rc = [
    "Sale Condition",
    "SalePrice",
    "Gr Liv Area"
]

dfx = dfx.loc[:, rc]

sns.lmplot(
    data = dfx,
    x = "Gr Liv Area",
    y = "SalePrice",
    hue = "Sale Condition",
    height = 15 
)

#### Does column "MS Zoning" has a relevant influence on the sale price?

In [ ]:
dfx = df_train.copy()

rc = [
    "MS Zoning",
    "SalePrice",
    "Gr Liv Area"
]

dfx = dfx.loc[:, rc]

f, ax = plt.subplots(figsize=(15, 10))
sns.scatterplot(
    data = dfx,
    x = "Gr Liv Area",
    y = "SalePrice",
    hue = "MS Zoning",
    ax = ax
)

In [ ]:
f, ax = plt.subplots(figsize=(15, 10))
sns.boxplot(
    data = dfx,
    x = "MS Zoning",
    y = "SalePrice",
    ax = ax
)

#### Does "Lot Area" is a good predictor of sale price?

In [ ]:
dfx = df_train.copy()

rc = [
    "Lot Area",
    "SalePrice",
    "MS Zoning"
]

dfx = dfx.loc[:, rc]

f, ax = plt.subplots(figsize=(15, 10))
sns.scatterplot(
    data = dfx,
    x = "Lot Area",
    y = "SalePrice",
    hue = "MS Zoning",
    ax = ax
)

# Data preparation

In [7]:
df_train_c = clean_data(df_train)
df_train_c

,MS Zoning,Lot Area,Exter Qual,Exter Cond,Gr Liv Area,Sale Condition
0,0,3072,Gd,TA,1548,Normal
2,1,3013,Gd,TA,1506,Normal
4,1,3196,Gd,TA,1456,Normal
6,1,3182,Gd,TA,1266,Normal
7,1,3203,Gd,TA,1145,Normal
...,...,...,...,...,...,...
1462,1,12000,Gd,TA,2362,Normal
1463,1,14694,Ex,Ex,1694,Normal
1464,1,19522,Gd,TA,2503,Normal
1465,1,9600,TA,TA,1262,Normal


---

# Notes

## Features dictionary

In [ ]:
features_dict

In [ ]:
for key in features_dict:
    features_dict[key]["notes"] = "-"

In [ ]:
features_dict

In [ ]:
json_dump_dict(features_dict)

---

## Test 1
- SkLearn Linear Regression (Housing Prices Example): https://www.youtube.com/watch?v=JTj-WgWLKFM

In [ ]:
boston = datasets.load_boston()

In [ ]:
boston

In [ ]:
df_x = pd.DataFrame(boston.data, columns=boston.feature_names)
df_y = pd.DataFrame(boston.target)

In [ ]:
df_x.describe()

In [ ]:
reg = linear_model.LinearRegression()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=4)

In [ ]:
reg.fit(x_train, y_train)

In [ ]:
reg.coef_

In [ ]:
a = reg.predict(x_test)

In [ ]:
a[4]

In [ ]:
y_test[0]

In [ ]:
# MSE
np.mean((a - y_test)**2)

---